In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [3]:
!chcp 1251

’ҐЄгй п Є®¤®ў п бва ­Ёж : 1251


In [15]:
!dir

 Том в устройстве E имеет метку Локальный диск
 Серийный номер тома: DA2A-2BF0

 Содержимое папки E:\University\11sem\nirs\NeuralNet\Cats_Dogs

31.03.2021  15:54    <DIR>          .
31.03.2021  15:54    <DIR>          ..
31.03.2021  15:45    <DIR>          .ipynb_checkpoints
31.03.2021  15:44         1 114 004 CatsDogs.ipynb
31.03.2021  15:41        33 627 912 cats_dogs_model.h5
30.03.2021  13:14       824 894 548 kagglecatsanddogs_3367a.zip
31.03.2021  15:54             7 327 Load_model.ipynb
31.03.2021  15:50                87 readme.md
31.03.2021  15:41    <DIR>          saved_epochs
31.03.2021  15:53        33 627 912 trained_model_cats_and_dogs.h5
               6 файлов    893 271 790 байт
               4 папок  88 345 403 392 байт свободно


In [16]:
model = keras.models.load_model('trained_model_cats_and_dogs.h5')

In [17]:
path_ = "C:/Users/corsair/.keras/datasets/kagglecatsanddogs_3367a/PetImages"
image_size = (180, 180)

In [18]:
img = keras.preprocessing.image.load_img(path_+
    "/Cat/6778.jpg", target_size=image_size
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
score = predictions[0]
print(
    "This image is %.2f percent cat and %.2f percent dog."
    % (100 * (1 - score), 100 * score)
)

This image is 99.92 percent cat and 0.08 percent dog.


### Узнать имена входного и выходного слоёв

In [27]:
[node.op.name for node in model.inputs]

['input_1']

In [26]:
[node.op.name for node in model.outputs]
#[print(n.name) for n in tf.get_default_graph().as_graph_def().node]

['dense/Sigmoid']

In [29]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 180, 180, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 180, 180, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 180, 180, 3)  0           sequential[0][0]                 
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 90, 90, 32)   896         rescaling[0][0]                  
______________________________________________________________________________________________

In [24]:
#full_model = tf.function(lambda x: model(x))
#full_model = full_model.get_concrete_function(tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

In [22]:
#min_graph = convert_variables_to_constants(session, session.graph_def, [node.op.name for node in model.outputs])

NameError: name 'convert_variables_to_constants' is not defined

### Conversion

In [21]:
import logging
import tensorflow as tf
from tensorflow.compat.v1 import graph_util
from tensorflow.python.keras import backend as K
from tensorflow import keras

# necessary !!!
tf.compat.v1.disable_eager_execution()

h5_path = 'trained_model_cats_and_dogs.h5'
model = keras.models.load_model(h5_path)
model.summary()
# save pb
with K.get_session() as sess:
    output_names = [out.op.name for out in model.outputs]
    input_graph_def = sess.graph.as_graph_def()
    for node in input_graph_def.node:
        node.device = ""
    graph = graph_util.remove_training_nodes(input_graph_def)
    graph_frozen = graph_util.convert_variables_to_constants(sess, graph, output_names)
    tf.io.write_graph(graph_frozen, 'trained_model_cats_and_dogs.pb', as_text=False)
logging.info("save pb successfully！")

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 180, 180, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 180, 180, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 180, 180, 3)  0           sequential[0][0]                 
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 90, 90, 32)   896         rescaling[0][0]                  
______________________________________________________________________________________________

Instructions for updating:
Use `tf.compat.v1.graph_util.remove_training_nodes`
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


FailedPreconditionError: 2 root error(s) found.
  (0) Failed precondition: Error while reading resource variable Variable_1 from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/Variable_1/class tensorflow::Var does not exist.
	 [[node Variable_1/Read/ReadVariableOp (defined at <ipython-input-21-206ff5b106ef>:11) ]]
	 [[batch_normalization_4/moving_mean/Read/ReadVariableOp/_1585]]
  (1) Failed precondition: Error while reading resource variable Variable_1 from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/Variable_1/class tensorflow::Var does not exist.
	 [[node Variable_1/Read/ReadVariableOp (defined at <ipython-input-21-206ff5b106ef>:11) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'Variable_1/Read/ReadVariableOp':
  File "C:\Anaconda3\envs\tf-2\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, Non

In [20]:


import tensorflow as tf
from tensorflow.python.keras import backend as K
from tensorflow import graph_util
from tensorflow.python.tools import optimize_for_inference_lib

sess = K.get_session()
outname = "output_node0"
tf.identity(model.outputs[0], name=outname)
constant_graph = graph_util.convert_variables_to_constants_v2(sess, sess.graph.as_graph_def(),[outname])
tf.train.write_graph(constant_graph, "./", "lenet.pb", as_text=False)

AttributeError: module 'tensorflow._api.v2.graph_util' has no attribute 'convert_variables_to_constants_v2'